Make a copy of this notebook and rename using your USERID in the following format, 2017CSZ8058

Give read access to keshavkolluru@gmail.com, vishalsaley114@gmail.com and kartikeya.badola@gmail.com


In [1]:
## DONT CHANGE THIS CELL
!wget http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/data.zip

--2021-10-11 13:44:30--  http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/data.zip
Resolving www.cse.iitd.ac.in (www.cse.iitd.ac.in)... 103.27.9.152
Connecting to www.cse.iitd.ac.in (www.cse.iitd.ac.in)|103.27.9.152|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/data.zip [following]
--2021-10-11 13:44:30--  https://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/data.zip
Connecting to www.cse.iitd.ac.in (www.cse.iitd.ac.in)|103.27.9.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 217313 (212K) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>] 212.22K  86.2KB/s    in 2.5s    

2021-10-11 13:44:34 (86.2 KB/s) - ‘data.zip’ saved [217313/217313]



In [2]:
!unzip data.zip

Archive:  data.zip
   creating: data/
   creating: data/train/
  inflating: data/train/train.gold.txt  
  inflating: data/train/train.data.txt  
   creating: data/validation/
  inflating: data/validation/validation.data.txt  
  inflating: data/validation/validation.gold.txt  


In [3]:
!pip install -U torchtext==0.10.0

In [4]:
## Import relevant packages

import os

import time
import datetime

import torch
import torch.optim as O
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR

import logging
from argparse import ArgumentParser

from pdb import set_trace
import pandas as pd
import torchtext
import dill
import random
import numpy as np
from torch.utils.data import Dataset
from torchtext.legacy.data import Field, TabularDataset, BucketIterator, LabelField
from torchtext.data.utils import get_tokenizer
import spacy

torch.manual_seed(4)
random.seed(4)
np.random.seed(4)


In [5]:
model_dir = './MODEL'
if os.path.exists(model_dir) == False:
  os.mkdir(model_dir)

In [6]:
def get_device(gpu_no):
	if torch.cuda.is_available():
		torch.cuda.set_device(gpu_no)
		return torch.device('cuda:{}'.format(gpu_no))
	else:
		return torch.device('cpu')	
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [7]:
def prepareDataset():
    x_train = pd.read_csv('/content/data/train/train.data.txt', sep = '\t',header = None, names = ["word","pos","P1P2" , "sentence1", "sentence2"])
    y_train = pd.read_csv('/content/data/train/train.gold.txt', header = None, names = ['labels'])
    y_train = y_train.labels.apply(lambda x: 0 if x == 'F' else 1)
    a = x_train["P1P2"].str.split(pat = "-",expand = True)
    a.columns = ['P1','P2']
    x_train = x_train.join(a)
    x_train = x_train.join(y_train)
    x_train.to_csv('TrainData.csv')

    x_dev = pd.read_csv('/content/data/validation/validation.data.txt', sep = '\t',header = None, names = ["word","pos","P1P2" , "sentence1", "sentence2"])
    y_dev= pd.read_csv('/content/data/validation/validation.gold.txt', header = None, names = ['labels'])
    y_dev = y_dev.labels.apply(lambda x: 0 if x == 'F' else 1)
    b = x_dev["P1P2"].str.split(pat = "-",expand = True)
    b.columns = ['P1','P2']
    x_dev = x_dev.join(b)
    x_dev = x_dev.join(y_dev)
    x_dev.to_csv('ValidationData.csv')
    
prepareDataset()

In [8]:
spacy_tokens = spacy.load('en')

def tokenizer(sentence):
    return [token.text for token in spacy_tokens.tokenizer(sentence)]

sentence = Field(sequential=True, use_vocab=True,  tokenize = tokenizer , lower = True)
position = Field(sequential=False,use_vocab=False)
labels = LabelField(sequential=False, use_vocab=False)

fields = {'sentence1' : ('sentence1' , sentence), 'sentence2' : ('sentence2' , sentence), 'word' : ('word' , sentence) , 'P1' : ('P1',position), 'P2' : ('P2',position), 'labels' : ('labels' , labels) }


train_data , dev_data = TabularDataset.splits(path = './' , train = 'TrainData.csv' , validation='ValidationData.csv' , format = 'csv' , fields= fields )
sentence.build_vocab(train_data, min_freq = 2,vectors = 'glove.6B.300d' )

train_iter , dev_iter = BucketIterator.splits((train_data,dev_data), batch_size=32 , device = 'cpu' , sort=False,repeat = False)
with open(os.path.join('MODEL','vocab'),'wb') as file:
  dill.dump(sentence,file)


.vector_cache/glove.6B.zip: 862MB [02:40, 5.37MB/s]                           
100%|█████████▉| 399999/400000 [00:45<00:00, 8857.63it/s]


In [9]:
class LSTMModel(nn.Module):
  def __init__(self, embed_dim, hidden, n_layers):
        super(LSTMModel, self).__init__()
        self.hidden = hidden
        self.n_layers = n_layers
        self.embed = nn.Embedding.from_pretrained(sentence.vocab.vectors, freeze=False)
        self.lstm = nn.LSTM(embed_dim, hidden,num_layers=n_layers,bidirectional=False)#batch_first = True #dropout=0.2
        
  def forward(self, input):
  
        input.sentence1 = input.sentence1.to(device)
        input.sentence2 = input.sentence2.to(device)
        input.P1 = input.P1.to(device)
        input.P2 = input.P2.to(device)
        embed_out = self.embed(input.sentence1).to(device)
        # embed_out = self.dropout(embed_out).to(device)
        hidden_in = torch.zeros(self.n_layers ,input.sentence1.shape[1], self.hidden).to(device)
        cell_in = torch.zeros( self.n_layers , input.sentence1.shape[1], self.hidden).to(device)
        out_lstm, (hidden_out, cell_out) = self.lstm(embed_out,(hidden_in, cell_in))
        
        
        
        embed_out2 = self.embed(input.sentence2).to(device)
        # embed_out2 = self.dropout(embed_out2).to(device)
        hidden_in2 = torch.zeros(self.n_layers  ,input.sentence2.shape[1], self.hidden).to(device)
        cell_in2 = torch.zeros( self.n_layers , input.sentence2.shape[1], self.hidden).to(device)
        out_lstm2, (hidden_out2, cell_out2) = self.lstm(embed_out2,(hidden_in2, cell_in2))

        
        
        dim_1 = out_lstm[input.P1,torch.arange(out_lstm.size(1))]
        dim_2 = out_lstm2[input.P2,torch.arange(out_lstm2.size(1))]
        
        cos = nn.CosineSimilarity(dim=1, eps=1e-8)
        output = cos(dim_1, dim_2)
        
        output = torch.sigmoid(output)

        return output

In [10]:
## Basic training loop

class Train():
	def __init__(self):
		self.lr = 0.0001
		self.epochs = 20
		self.gpu = 0
		self.device = get_device(self.gpu)
		self.step_size = 5
		self.gamma = 0.6
		self.vocab_size = len(sentence.vocab)
		self.emb_dim = sentence.vocab.vectors.shape[1]
		self.hidden = 512
		self.layers = 2
		
		self.train_iter = train_iter
		self.dev_iter = dev_iter
		
		self.model = LSTMModel(self.emb_dim,self.hidden,self.layers)
		
		self.model.to(self.device)
		self.criterion = nn.BCELoss()
		self.opt = O.Adam(self.model.parameters(), lr = self.lr)
		self.best_val_acc = None
		self.scheduler = StepLR(self.opt, step_size=self.step_size, gamma=self.gamma)

		

	def result_checkpoint(self, epoch, train_loss, val_loss, train_acc, val_acc, took):
		if self.best_val_acc is None or val_acc > self.best_val_acc:
			self.best_val_acc = val_acc
			torch.save({
				'accuracy': self.best_val_acc,
				'model_dict': self.model.state_dict(),
			}, './MODEL/Model.pt')
	
	def train(self):
		self.model.train(); self.train_iter.init_epoch()
		n_correct, n_total, n_loss = 0, 0, 0
		
		for batch_idx, batch in enumerate(train_iter):
			self.opt.zero_grad()
			answer = self.model(batch)
			loss = self.criterion(answer, batch.labels.float().to(device))
			n_correct += ((answer>=0.5).int() == batch.labels.to(device)).float().sum()
			n_total += batch.batch_size
			n_loss += loss.item()
			loss.backward(); self.opt.step()
		train_loss = n_loss/n_total
		train_acc = 100. * n_correct/n_total
		return train_loss, train_acc

	def validate(self):
		self.model.eval(); self.dev_iter.init_epoch()
		n_correct, n_total, n_loss = 0, 0, 0
		with torch.no_grad():
			for batch_idx, batch in enumerate(self.dev_iter):
				answer = self.model(batch)
				loss = self.criterion(answer, batch.labels.float().to(device))
				n_correct += ((answer>=0.5).int() == batch.labels.to(device)).float().sum()
				n_total += batch.batch_size
				n_loss += loss.item()

			val_loss = n_loss/n_total
			val_acc = 100. * n_correct/n_total
			return val_loss, val_acc

	def execute(self):
		print(" [*] Training starts!")
		print('-' * 99)
		for epoch in range(1, self.epochs+1):
			start = time.time()

			train_loss, train_acc = self.train()
			val_loss, val_acc = self.validate()
			self.scheduler.step()
			
			took = time.time()-start
			self.result_checkpoint(epoch, train_loss, val_loss, train_acc, val_acc, took)

			print('| Epoch {:3d} | train loss {:5.5f} | train acc {:5.5f} | val loss {:5.5f} | val acc {:5.5f} | time: {:5.2f}s |'.format(
				epoch, train_loss, train_acc, val_loss, val_acc, took))
		self.finish()

	def finish(self):
		print("[*] Training finished!\n\n")
		print('-' * 99)
		print(" [*] Training finished!")
		print(" [*] Please find the saved model and training log in results_dir")


In [11]:
task = Train()
task.execute()

 [*] Training starts!
---------------------------------------------------------------------------------------------------
| Epoch   1 | train loss 0.02200 | train acc 53.37141 | val loss 0.02140 | val acc 56.26959 | time: 144.05s |
| Epoch   2 | train loss 0.02080 | train acc 61.82756 | val loss 0.02103 | val acc 58.93417 | time: 141.15s |
| Epoch   3 | train loss 0.02018 | train acc 64.90420 | val loss 0.02113 | val acc 60.03135 | time: 140.95s |
| Epoch   4 | train loss 0.01966 | train acc 68.23876 | val loss 0.02082 | val acc 61.12852 | time: 142.49s |
| Epoch   5 | train loss 0.01916 | train acc 70.76271 | val loss 0.02088 | val acc 62.22570 | time: 140.33s |
| Epoch   6 | train loss 0.01861 | train acc 73.78408 | val loss 0.02085 | val acc 61.12852 | time: 141.51s |
| Epoch   7 | train loss 0.01832 | train acc 75.64481 | val loss 0.02091 | val acc 60.50157 | time: 141.44s |
| Epoch   8 | train loss 0.01805 | train acc 76.25276 | val loss 0.02091 | val acc 60.34483 | time: 140.41s 

In [12]:
## Zip the final model and all the required files, such as vocabulary
# Replace USERID with your own, such as 2017CSZ8058
!zip -r 2018EE10434_A_model.zip MODEL

## Upload it to Google drive and ensure that the testing notebook uses the correct link

  adding: MODEL/ (stored 0%)
  adding: MODEL/vocab (deflated 11%)
  adding: MODEL/Model.pt (deflated 7%)
